In [59]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
dataset_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
dataset_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [21]:
dataset = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

X1, y = dataset.data, dataset.target
vectorizer = HashingVectorizer(stop_words='english', non_negative=True)
X = vectorizer.transform(X1)
features = features = dataset.target_names

/Users/shahd/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [74]:
import numpy
import sklearn.svm
from sklearn.feature_extraction.text import HashingVectorizer ,TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups 
from sklearn.naive_bayes import MultinomialNB

def reduce_features(solution, features):
    selected_elements_indices = numpy.where(solution == 1)[0]
    reduced_features = features[:, selected_elements_indices]
    return reduced_features


def classification_accuracy(labels, predictions):
    correct = numpy.where(labels == predictions)[0]
    accuracy = correct.shape[0]/labels.shape[0]
    return accuracy


def cal_pop_fitness(pop, features, labels, train_indices, test_indices, indices):
    accuracies = numpy.zeros(pop.shape[0])
    idx = 0

    for curr_solution in pop:
        reduced_features = reduce_features(curr_solution, features)
        train_data = reduced_features[train_indices, :]
#         print ("train_data: ", train_data.shape)
        test_data = reduced_features[test_indices, :]
#         print ("test_data: ", test_data.shape)

        train_labels = labels[train_indices]
        
        test_labels = labels[test_indices]
        data_labels= labels[indices]
        data = reduced_features[indices, :]
        SV_classifier = MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)
#         SV_classifier.fit(train_data, train_labels)
#         sklearn.svm.SVC(gamma='scale')
#         SV_classifier.fit(X=train_data, y=train_labels)

#         predictions = SV_classifier.predict(test_data)
#         accuracies[idx] = classification_accuracy(test_labels, predictions)
#         print(features)
        accuracies[idx] = cross_val_score(SV_classifier, data, data_labels, cv=5, scoring='accuracy').mean()
        idx = idx + 1
    return accuracies

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents


def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring


def mutation(offspring_crossover, num_mutations=2):
    mutation_idx = numpy.random.randint(low=0, high=offspring_crossover.shape[1], size=num_mutations)
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        offspring_crossover[idx, mutation_idx] = 1 - offspring_crossover[idx, mutation_idx]
    return offspring_crossover

In [ ]:
import numpy
import pickle
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot
from sklearn.naive_bayes import MultinomialNB 

cats =['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware']
dataset = fetch_20newsgroups(subset='all', categories=cats)

X1, y = dataset.data, dataset.target
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.3, stop_words='english',smooth_idf =True)
# vectorizer = HashingVectorizer(stop_words='english', non_negative=True)
X = vectorizer.fit_transform(X1)
# features = dataset.target_names
data_outputs = dataset.target
data_inputs = X
est = MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)
# tttt =cross_val_score(est, X, y, cv=5, scoring='accuracy').mean()
print ("MultinomialNB 10-Cross Validation :",cross_val_score(est, data_inputs, data_outputs, cv=5, scoring='accuracy').mean())


# f = open("dataset_features.pkl", "rb")
# data_inputs = pickle.load(f)
# f.close()

# f = open("outputs.pkl", "rb")
# data_outputs = pickle.load(f)
# f.close()


num_samples = data_inputs.shape[0]
num_feature_elements = data_inputs.shape[1]

train_indices = numpy.arange(1, num_samples, 4)
test_indices = numpy.arange(0, num_samples, 4)
indices = numpy.arange(0, num_samples, 2)
print("Number of training samples: ", train_indices.shape[0])
print("Number of test samples: ", test_indices.shape[0])

"""
Genetic algorithm parameters:
    Population size
    Mating pool size
    Number of mutations
"""
sol_per_pop = 900 # Population size.
num_parents_mating = 300 # Number of parents inside the mating pool.
num_mutations = 30 # Number of elements to mutate.

# Defining the population shape.
pop_shape = (sol_per_pop, num_feature_elements)

# Creating the initial population.
new_population = numpy.random.randint(low=0, high=2, size=pop_shape)
print(new_population.shape)

best_outputs = []
num_generations = 50
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population, data_inputs, data_outputs, train_indices, test_indices, indices)

    best_outputs.append(numpy.max(fitness))
    # The best result in the current iteration.
    print("Best result : ", best_outputs[-1])

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, offspring_size=(pop_shape[0]-parents.shape[0], num_feature_elements))

    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover, num_mutations=num_mutations)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

# Getting the best solution after iterating finishing all generations.
# At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population, data_inputs, data_outputs, train_indices, test_indices, indices)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))[0]
best_match_idx = best_match_idx[0]

best_solution = new_population[best_match_idx, :]
best_solution_indices = numpy.where(best_solution == 1)[0]
best_solution_num_elements = best_solution_indices.shape[0]
best_solution_fitness = fitness[best_match_idx]
est = MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)
tttt =cross_val_score(est, X[:,best_solution], y, cv=5, scoring='accuracy').mean()
print ("MultinomialNB 10-Cross Validation accuracy:",tttt)

print("best_match_idx : ", best_match_idx)
print("best_solution : ", best_solution)
print("Selected indices : ", best_solution_indices)
print("Number of selected elements : ", best_solution_num_elements)
print("Best solution fitness : ", best_solution_fitness)

matplotlib.pyplot.plot(best_outputs)
matplotlib.pyplot.xlabel("Iteration")
matplotlib.pyplot.ylabel("Fitness")
matplotlib.pyplot.show()
# 0.9232425531812523
# 0.9112133005119546
# 0.911742689829231
# 0.878171970381991
# 0.8925071997924384

MultinomialNB 10-Cross Validation : 0.9232425531812523
Number of training samples:  935
Number of test samples:  935
(900, 82269)
Generation :  0
Best result :  0.8935796166837233
Generation :  1
Best result :  0.8935796166837233
Generation :  2
Best result :  0.8935796166837233
Generation :  3
Best result :  0.8935796166837233
Generation :  4
Best result :  0.8935853669083414
Generation :  5
Best result :  0.8935853669083414
Generation :  6
Best result :  0.8935853669083414
Generation :  7
Best result :  0.8935853669083414
Generation :  8
Best result :  0.8935853669083414
Generation :  9
Best result :  0.8935853669083414
Generation :  10
Best result :  0.8935853669083414
Generation :  11
Best result :  0.8935853669083414
Generation :  12
Best result :  0.8941187002416747
Generation :  13
Best result :  0.8941187002416747
Generation :  14


In [ ]:
print(best_solution)

sol_per_pop = 400 # Population size.
num_parents_mating = 200 # Number of parents inside the mating pool.
num_mutations = 30 # Number of elements to mutate
0.8952199143808468

In [ ]:
0.8727272727272727
0.8716577540106952
0.8641711229946524
0.8695187165775401
0.8673796791443851
0.8663101604278075


0.8673796791443851
    
0.6186332767402377


-------------------tdidf
---------------- 
num_generations = 50    
sol_per_pop = 100 # Population size.
num_parents_mating = 50 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
result: 0.8673796791443851
    
---------------- 
num_generations = 50      
sol_per_pop = 100 # Population size.
num_parents_mating = 50 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8695187165775401
---------------- 
num_generations = 50  
sol_per_pop = 200 # Population size.
num_parents_mating = 50 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8663101604278075
---------------- 
num_generations = 100
sol_per_pop = 200 # Population size.
num_parents_mating = 50 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8695187165775401
---------------- 
num_generations = 100
sol_per_pop = 200 # Population size.
num_parents_mating = 50 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8673796791443851
---------------- 
num_generations = 100
sol_per_pop = 300 # Population size.
num_parents_mating = 50 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8737967914438503
----------------
num_generations = 100
sol_per_pop = 300 # Population size.
num_parents_mating = 100 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8663101604278075
----------------
num_generations = 100
sol_per_pop = 300 # Population size.
num_parents_mating = 100 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8737967914438503
----------------
num_generations = 100
sol_per_pop = 300 # Population size.
num_parents_mating = 30 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8673796791443851

0.8695187165775401
----------------
num_generations = 100
sol_per_pop = 300 # Population size.
num_parents_mating = 150 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8684491978609625
------------------
num_generations = 100
sol_per_pop = 500 # Population size.
num_parents_mating = 100 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8695187165775401
------------------
num_generations = 100
sol_per_pop = 500 # Population size.
num_parents_mating = 200 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8759358288770054
----------------
sol_per_pop = 500 # Population size.
num_parents_mating = 200 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8759358288770054
------------------
num_generations = 100
sol_per_pop = 500 # Population size.
num_parents_mating = 200 # Number of parents inside the mating pool.
num_mutations = 30 # Number of elements to mutate.
0.8748663101604278
------------------
num_generations = 100
sol_per_pop = 500 # Population size.
num_parents_mating = 200 # Number of parents inside the mating pool.
num_mutations = 30 # Number of elements to mutate.
0.8748663101604278
------------------
num_generations = 100
sol_per_pop = 900 # Population size.
num_parents_mating = 250 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
0.8770053475935828

In [12]:
from sklearn import metrics
vectorizer = TfidfVectorizer(stop_words='english')
# (sublinear_tf=True, max_df=0.3, stop_words='english',smooth_idf =True)

dataset_train_vectors = vectorizer.fit_transform(dataset_train.data)
dataset_test_vectors = vectorizer.transform(dataset_test.data) 

Xtr = dataset_train_vectors
ytr = dataset_train.target
Xtt = dataset_test_vectors
ytt = dataset_test.target

# Instantiate the estimator
clf_MNB = MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)

# Fit the model with data (aka "model training")
clf_MNB.fit(Xtr, ytr)

# Predict the response for a new observation
y_pred_mnb = clf_MNB.predict(Xtt)
# print ("Predicted Class Labels:",y_pred_mnb)

# calculate accuracy
print ("Classification Accuracy:",metrics.accuracy_score(ytt, y_pred_mnb))

NameError: name 'dataset_train' is not defined

In [34]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

cats =['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware']

dataset_train = fetch_20newsgroups(subset='train', categories=cats)
dataset_test = fetch_20newsgroups(subset='test', categories=cats)
data = fetch_20newsgroups(subset='all', categories=cats)

vectorizer = TfidfVectorizer(stop_words='english')
# TfidfVectorizer(sublinear_tf=True, max_df=0.3, stop_words='english',smooth_idf =True)

dataset_train_vectors = vectorizer.fit_transform(dataset_train.data)
dataset_test_vectors = vectorizer.transform(dataset_test.data) 
dataset_vectors = vectorizer.fit_transform(data.data)
yy = data.target
XX = dataset_vectors

Xtr = dataset_train_vectors
ytr = dataset_train.target
Xtt = dataset_test_vectors
ytt = dataset_test.target

# Instantiate the estimator
clf_MNB = MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)

# Fit the model with data (aka "model training")
clf_MNB.fit(Xtr, ytr)

# Predict the response for a new observation
y_pred_mnb = clf_MNB.predict(Xtt)
print ("Predicted Class Labels:",y_pred_mnb)

# calculate accuracy
print ("Classification Accuracy:",metrics.accuracy_score(ytt, y_pred_mnb))
y_pred_mnb = clf_MNB.predict(Xtt)
# print ("Predicted Class Labels:",y_pred_mnb)


print ("MultinomialNB 10-Cross Validation :",cross_val_score(clf_MNB, XX, yy, cv=5, scoring='accuracy').mean())

Predicted Class Labels: [1 2 1 ... 1 3 3]
Classification Accuracy: 0.856760374832664
MultinomialNB 10-Cross Validation : 0.9187038860806073


In [ ]:
TfidfVectorizer:
Classification Accuracy: 0.856760374832664
Predicted Class Labels: [1 2 1 ... 1 3 3]
MultinomialNB 10-Cross Validation : 0.9187038860806073

In [ ]:
sol_per_pop = 20: 0.5952886247877759
----------------------
sol_per_pop = 8 # Population size.
num_parents_mating = 4 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
num_generations = 100
result:0.5988964346349746
---------------------
sol_per_pop = 200 # Population size.
num_parents_mating = 4 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
num_generations = 100
result:0.6050509337860781
---------------------
result:0.6118421052631579
--------------------- ✔️
num_generations = 100
sol_per_pop = 50 # Population size. ✔️
num_parents_mating = 4 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
results:0.6169354838709677
--------------------- 
num_generations = 100
sol_per_pop = 20 # Population size.
num_parents_mating = 4 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
results:0.6143887945670629
---------------------
num_generations = 100
sol_per_pop = 20 # Population size.
num_parents_mating = 10 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
result: 0.6137521222410866
---------------------
num_generations = 100
sol_per_pop = 50 # Population size.
num_parents_mating = 25 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
result:0.6177843803056027
---------------------
num_generations = 50
sol_per_pop = 500 # Population size.
num_parents_mating = 250 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
result:0.62330220713073
--------------------
num_generations = 50
sol_per_pop = 500 # Population size.
num_parents_mating = 250 # Number of parents inside the mating pool.
num_mutations = 30 # Number of elements to mutate.
result: 0.6205432937181664
--------------------
num_generations = 50
sol_per_pop = 5000 # Population size.
num_parents_mating = 2500 # Number of parents inside the mating pool.
num_mutations = 10 # Number of elements to mutate.
result: 0.6226655348047538
    
------------------
------------------
num_generations = 50
sol_per_pop = 1000 # Population size.
num_parents_mating = 600 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
resultt: 0.8434163701067615
------------------
num_generations = 50
sol_per_pop = 1000 # Population size.
num_parents_mating = 500 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
resultt: 0.0.8380782918149466
------------------
num_generations = 50
sol_per_pop = 500 # Population size.
num_parents_mating = 250 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
resultt: 0.0.8380782918149466    
0.8416370106761566
------------------
num_generations = 50
sol_per_pop = 100 # Population size.
num_parents_mating = 50 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
result: 0.8416370106761566
------------------
num_generations = 50
sol_per_pop = 100 # Population size.
num_parents_mating = 25 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
------------------
num_generations = 50
sol_per_pop = 50 # Population size.
num_parents_mating = 25 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
result:0.8673796791443851
------------------
num_generations = 50    
sol_per_pop = 500 # Population size.
num_parents_mating = 10 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
result: 0.8641711229946524
0.8695187165775401
------------------
num_generations = 50 